# Querying Flywheel for the June 30th, 2021 Datafreeze
**Project:** ExtraLong <br>
**Author:** Katja Zoner <br>
**Date:** 07/30/2021 <br>

In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import flywheel

from IPython.display import Markdown as md


In [2]:
logging.basicConfig(
    stream=sys.stdout, 
    level=logging.INFO, 
    format='%(levelname)s: %(message)s')

logger = logging.getLogger('Query Flywheel')

logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s: %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [3]:
# # Get API_KEY from FW profile
API_KEY = "upenn.flywheel.io:47vhOSDkwMxGRNxFq0"
PROJECT_LABEL = "ExtraLong"

# Get client
logger.info("Connecting to Flywheel Client...")
fw = flywheel.Client(API_KEY)
assert fw, "Your Flywheel CLI credentials aren't set!"

# Get project object
project = fw.projects.find_first(            
    'label="{}"'.format(PROJECT_LABEL))
assert project, "Project not found!"
logger.debug('Found project: %s (%s)',
                project['label'], project.id)

INFO: Connecting to Flywheel Client...


INFO: Connecting to Flywheel Client...


/Users/kzoner/.conda/envs/flywheel/lib/python3.8/site-packages/flywheel/flywheel.py:6274: UserWarning: Client version 15.3.1 does not match server version 14.6.4. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


# Step 1. Create superset dataframe.

1. Load scan superset csv into pandas dataframe.
2. Sort by date of scan.
3. Load in scanid -> sesid mapping csv.
4. Add sesid col to superset df.

In [5]:
# Read superset csv.
superset_csv = "csv/all_long_scans_oracle_cleaned.csv"
superset = pd.read_csv(superset_csv)

# Convert dates to timestamps for easy sorting.
superset["doscan"] = pd.to_datetime(superset["doscan"])

# Convert scanid to int
superset[["scanid","bblid"]] = superset[["scanid","bblid"]].astype("int64")

# Sort df by date.
superset.sort_values("doscan", inplace=True)

# Rearrange superset df for readability.
superset = superset[["scanid", "bblid", "scan_protocol", "doscan",  "timepoint", "ntimepoints", "scanage_months", "sex", "race", "ethnic", "scanstat"]]
superset
#superset.to_csv("all_long_scans_oracle_cleaned.csv",index=False)

,scanid,bblid,scan_protocol,doscan,timepoint,ntimepoints,scanage_months,sex,race,ethnic,scanstat
0,613,11660,700205 - Worden,2003-05-23,1,6,1240.0,NaN,NaN,NaN,C2
1,615,11518,700205 - Worden,2003-05-27,1,2,356.0,2.0,2.0,NaN,C2
2,597,11382,700205 - Worden,2003-07-17,1,4,260.0,2.0,2.0,2.0,C2
3,598,11565,700205 - Worden,2003-07-24,1,7,291.0,1.0,1.0,2.0,I7
4,599,11530,700205 - Worden,2003-07-29,1,4,331.0,1.0,2.0,2.0,C2
...,...,...,...,...,...,...,...,...,...,...,...
4253,11803,132782,842909 - TRANSCENDS_D1,2021-07-12,3,6,307.0,1.0,1.0,2.0,NaN
4251,11821,20082,843329 - LongGluCEST,2021-07-12,3,3,243.0,2.0,1.0,2.0,NaN
4254,11804,132782,842909 - TRANSCENDS_D1,2021-07-19,4,6,307.0,1.0,1.0,2.0,NaN
4255,11805,132782,842909 - TRANSCENDS_D1,2021-07-30,5,6,307.0,1.0,1.0,2.0,NaN


In [22]:
# Read in mapping csv
mapping_csv = "csv/scanid_to_seslabel_10-16-2019.csv"
mapping = pd.read_csv(mapping_csv)

# Rename seslabel col to sesid
mapping.columns = ["study", "bblid", "scanid", "sesid"]
mapping = mapping[["scanid", "bblid", "study", "sesid", ]]

# Cast col datatypes
mapping[["bblid", "scanid"]]=mapping[["bblid", "scanid"]].astype("int64")
mapping[["sesid","study"]]=mapping[["sesid","study"]].astype("string")

mapping = mapping.sort_values(["study","bblid","scanid"])
mapping.head(20)
mapping = mapping.rename(columns={"study": "project"})

# Step 2. Get scans already in ExtraLong project from previous datafreeze. Load into pandas dataframe.


In [8]:
xl_dict = {
    "bblid":[], 
    "sesid":[]
}

# Loop through subjects in project
for sub in project.subjects():
    # Loop through sessions in subject
    for ses in sub.sessions():
        ses = ses.reload()
        xl_dict["bblid"].append(sub.label.strip("sub-"))
        xl_dict["sesid"].append(ses.label.strip("ses-"))

# Create pandas dataframe
# xl = pd.DataFrame.from_dict(xl_dict)

# Step 3. Clean and prep dataframes for merge.

In [23]:
xl = pd.DataFrame.from_dict(xl_dict)
xl.columns = ["bblid", "sesid"]

xl["bblid"] = xl["bblid"].apply(lambda id: id.strip("sub-"))
xl["sesid"] = xl["sesid"].apply(lambda id: id.strip("ses-"))

xl["sesid"]=xl["sesid"].astype("string")
xl["bblid"]=xl["bblid"].astype("int64")

xl = xl.merge(mapping).sort_values(["project", "bblid", "scanid"])
xl = xl[["scanid", "bblid", "project", "sesid"]]

# Step 4. Compare ExtraLong contents to superset

In [30]:
xl.reset_index(drop=True, inplace=True)
superset.reset_index(drop=True, inplace=True)

xl_merged=pd.merge(xl,superset).sort_values("doscan")
#test = test[["scanid", "bblid", "study", "sesid"]]
xl_merged = xl_merged[[
    "bblid", "scanid", "sesid", "scan_protocol", 
    "project", "doscan", "timepoint", "ntimepoints", 
    "scanage_months", "sex", "race", "ethnic"
]]
xl_merged

,bblid,scanid,sesid,scan_protocol,project,doscan,timepoint,ntimepoints,scanage_months,sex,race,ethnic
44,130896,2542,AGGY1,808689 - AGGY,AGGY_808689,2009-10-28,1,6,144.0,2.0,2.0,2.0
850,80812,2646,PNC1,810336 - Big GO,PNC_CS_810336,2009-12-05,1,2,247.0,2.0,2.0,2.0
851,80854,2675,PNC1,810336 - Big GO,PNC_CS_810336,2009-12-17,1,2,242.0,1.0,1.0,2.0
873,82232,2706,PNC1,810336 - Big GO,PNC_CS_810336,2010-01-07,1,2,228.0,2.0,1.0,2.0
877,82458,2728,PNC1,810336 - Big GO,PNC_CS_810336,2010-01-09,1,2,229.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
520,98535,11254,11254,822831 - GRMPY,GRMPY_822831,2019-05-07,3,4,221.0,1.0,1.0,1.0
561,114723,11256,11256,822831 - GRMPY,GRMPY_822831,2019-05-09,2,2,328.0,1.0,2.0,2.0
596,126921,11262,11262,822831 - GRMPY,GRMPY_822831,2019-05-15,2,3,226.0,1.0,2.0,2.0
648,112200,11349,motive1,829502 - MOTIVE,MOTIVE,2019-08-07,6,7,261.0,1.0,1.0,2.0


In [32]:
def filterBySessionCount(df, thresh):

    # Get scan counts for each subject (bblid).
    bblid_counts = df.groupby(["bblid"]).size().reset_index()
    bblid_counts.columns = ["bblid", "scan_count"]
    bblid_counts.sort_values("scan_count")

    # Get list of bblids that should be included
    include_bblids = bblid_counts.bblid[bblid_counts["scan_count"]>=thresh]

    # Filter dataframe to only include subjects that meet session requirements
    df = df[df["bblid"].isin(include_bblids)].copy()

    # Update ntimepoints column to indicate number of sessions in ExtraLong 2021 for each subject.
    for bblid in df.bblid.unique():

        # Get subject's number of timepoints in ExtraLong 2021
        num_tps = bblid_counts[bblid_counts.bblid == bblid].scan_count.item()

        # Update ntimeponts in ExtraLong dataframe
        df.loc[df.bblid==bblid, 'ntimepoints'] = num_tps

    # Update timepoint column to indicate timepoint number for each subject
    df["timepoint"] = df.groupby("bblid").cumcount()+1

    return df.sort_values(["bblid","doscan"])

In [34]:
import string

# Clean the sesid field for the entire dataframe
for index,scan in xl_merged.iterrows():
    
    sesid = str(scan.sesid)
    # Check for existing ses label
    
    # All numeric sesid's from old ExtraLong should be GRMPY
    if sesid.isdecimal():
        new = "GRMPY"
    # Change motive to all caps to match

    elif sesid == 'motive1':
        new = "MOTIVE"
    
    elif sesid.lower() == 'nan':
        new = SESSION_LABEL_DICT[scan.project]
    
    else:
        new = sesid.rstrip(string.digits)
    
    xl_merged.loc[index, "sesid"] = new

xl_merged = filterBySessionCount(xl_merged, 2)
xl_merged.to_csv("csv/ExtraLong-Datafreeze-2019.csv", index=False)
xl_merged


,bblid,scanid,sesid,scan_protocol,project,doscan,timepoint,ntimepoints,scanage_months,sex,race,ethnic
269,10410,6776,FNDM,810211 - FNDM,FNDM1_810211,2012-04-24,1,2,556.0,2.0,2.0,2.0
314,10410,6843,FNDM,810211 - FNDM,FNDM2_810211,2012-05-11,2,2,557.0,2.0,2.0,2.0
211,11186,3395,DAY,808799 - DAY2,DAY2_808799,2010-06-10,1,2,534.0,1.0,1.0,2.0
315,11186,6378,FNDM,810211 - FNDM,FNDM2_810211,2012-02-03,2,2,554.0,1.0,1.0,2.0
212,11242,3360,DAY,808799 - DAY2,DAY2_808799,2010-06-02,1,2,694.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
622,139272,10109,GRMPY,822831 - GRMPY,GRMPY_822831,2016-04-07,6,6,259.0,2.0,2.0,2.0
1484,139490,8461,PNC,810336 - Big GO,PNC_CS_810336,2013-08-30,1,2,105.0,1.0,2.0,2.0
210,139490,10564,CONTE,815814 - Conte,CONTE_815814,2017-04-29,2,2,149.0,1.0,2.0,2.0
1485,139553,8410,PNC,810336 - Big GO,PNC_CS_810336,2013-08-23,1,2,107.0,2.0,2.0,2.0


In [371]:
#xl.set_index("scanid", inplace=True)
#superset.set_index("scanid",inplace=True)
pd.merge(xl,superset).sort_values("doscan")

xl[~xl.scanid.isin(superset.scanid)]
xl[~xl.scanid.isin(test.scanid)]

# The following were in ExtraLong but are not in the superset csv.

,scanid,bblid,study,sesid
510,11191,95257,GRMPY_822831,11191
671,9817,17491,NEFF_818028,NEFF2


In [434]:
def exportScansByProtocol(df,csvName):
    pDict = {
        'scan_protocol': [],
        'scan_count': [],
        'first_scan': [],
        'last_scan': []
        }

    # Helper function to find timepoints first and last scans for a given protocol.
    def getFirstLast(df, protocol):
        pSorted = df[df.scan_protocol == protocol].sort_values('doscan')
        first = pSorted.iloc[0].doscan
        last = pSorted.iloc[-1].doscan
        return (first, last)

    # Get df of scan count per protocol
    scans_by_protocol = df.groupby("scan_protocol").size().reset_index()
    scans_by_protocol.columns = ["scan_protocol", "scan_count"]

    # For each protocol, find timestamp of first and last scans, add to dict.
    for index, row in scans_by_protocol.iterrows():
        count = row.scan_count
        protocol=row.scan_protocol
        first, last = getFirstLast(df, protocol)
        pDict['scan_protocol'].append(protocol)
        pDict['scan_count'].append(count)
        pDict['first_scan'].append(first)
        pDict['last_scan'].append(last)

    # Replace scans_by_protocol with of: scan_protocol, scan_count, AND first_scan, last_scan
    scans_by_protocol = pd.DataFrame.from_dict(pDict)

    # Export to csv 
    scans_by_protocol.sort_values(["first_scan","last_scan"]).to_csv(csvName,index=False)


In [438]:
exportScansByProtocol(superset,"superset_scan_counts_by_protocol.csv")
exportScansByProtocol(xl_merged,"extralong_scan_counts_by_protocol.csv") 

In [364]:
# Get mapping of study name (Flywheel) to scan_protocol field in superset csv.
study_to_protocol_map = test.groupby(["study", "scan_protocol"]).size().reset_index()
study_to_protocol_map

,study,scan_protocol,0
0,AGGY_808689,808689 - AGGY,49
1,CONTE_815814,815814 - Conte,162
2,DAY2_808799,808799 - DAY2,58
3,FNDM1_810211,810211 - FNDM,45
4,FNDM2_810211,810211 - FNDM,109
5,GRMPY_822831,810336 - GO3 FOLLOW UP,9
6,GRMPY_822831,810336 - Go3,6
7,GRMPY_822831,822831 - GRMPY,185
8,MOTIVE,829502 - MOTIVE,34
9,NEFF_818028,818028 - Effort,27


In [ ]:
# Get scans where scan_protocol==
sp = '810336 - Go3'
proj='GRMPY_822831'

AGGY_808689
CONTE_815814
DAY2_808799
FNDM2_810211
NEFF_818028
NODRA_816281
ONM_816275
PNC_LG_810336




# Step 5. Get set of all scans that should be included in new data freeze.
## 2021 Data Freeze Requirements:
- From project that was recruited from PNC (check list with Kosha)
- Subject has been seen 2+ times

In [286]:
# 1. Filter by study (only include projects recruited fronm PNC)??
pnc_set = superset[superset["scan_protocol"].isin(protocols)]

# 2. Filter by subject's session count (only include subjects with 2+ scans).
bblid_counts = pnc_set.groupby(["bblid"]).size().reset_index().sort_values(0)
include_bblids = bblid_counts[bblid_counts[0]>1]
include_bblids

,bblid,0
611,100873,2
394,87905,2
587,98902,2
429,89627,2
608,100482,2
...,...,...
469,91919,10
656,105176,10
320,83835,11
846,121407,13


In [267]:
# For comparison, bblid counts currently in ExtraLong
xl.groupby(["bblid"]).size().reset_index().sort_values(0)

,bblid,0
0,10410,2
471,99032,2
469,98975,2
468,98902,2
466,98849,2
...,...,...
548,106573,7
702,121407,8
217,83835,8
528,105176,8


In [353]:
new_freeze = pnc_set[pnc_set["bblid"].isin(include_bblids.bblid)] 
new_freeze


Number of subjects:     860
Number of scans:        2623


# Step 6. Export new datafreeze as csv containing: `bblid`, `scanid`, `scan_protocol`, `timestamp`.

## Question: How do I get Flywheel project name from scan_protocol?
Seems to be that there are some mismatched / duplicate matched study-scan_protocol mappings.

# `Work for Presentation Below`

# ExtraLong Dataset from 2019 Data Freeze <br>
## `821` Subjects
## `2341` Scans

### Scans Per Study in ExtraLong 2019:

In [356]:
#print(
#    f"Number of subjects:     {len(new_set.bblid.unique())}\n"
#    f"Number of scans:        {len(new_set)}"
#)

# Number of scans by study.
xl_studies = xl.set_index(["study","bblid","sesid"]).count(level="study").reset_index()
xl_studies.columns = ["Study", "Scan Count"]
xl_studies.style.set_table_attributes("style='display:inline'").set_caption("Scans Per Study in ExtraLong 2019").hide_index()

Study,Scan Count
AGGY_808689,49
CONTE_815814,162
DAY2_808799,58
FNDM1_810211,45
FNDM2_810211,109
GRMPY_822831,201
MOTIVE,34
NEFF_818028,28
NODRA_816281,63
ONM_816275,97


# ExtraLong Dataset for 2021 Data Freeze
## `968` Subjects
## `3121` Scans

In [357]:
print(
    f"Number of subjects:     {len(new_freeze.bblid.unique())}\n"
    f"Number of scans:        {len(new_freeze)}"
)

# Number of scans by study.
#new_studies = new_set.set_index(["study","bblid","sesid"]).count(level="study").reset_index()
#new_studies.columns = ["Study", "Scan Count"]
#new_studies.style.set_table_attributes("style='display:inline'").set_caption("Scans Per Study in ExtraLong 2021").hide_index()

Number of subjects:     860
Number of scans:        2623
